## Interaktív Phi 3 Mini 4K Instruct Chatbot a Whisperrel

### Bevezetés:
Az Interaktív Phi 3 Mini 4K Instruct Chatbot egy eszköz, amely lehetővé teszi a felhasználók számára, hogy szöveges vagy hangbemenet segítségével kapcsolatba lépjenek a Microsoft Phi 3 Mini 4K instruct demóval. A chatbot különféle feladatokra használható, például fordításra, időjárás-frissítésekre és általános információgyűjtésre.


In [ ]:
#Install required Python Packages
!pip install accelerate
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install flash-attn --no-build-isolation', env={'FLASH_ATTENTION_SKIP_CUDA_BUILD': "TRUE"}, shell=True
!pip install transformers
!pip install wheel
!pip install gradio
!pip install pydub==0.25.1
!pip install edge-tts
!pip install openai-whisper==20231117
!pip install ffmpeg==1.4
# from IPython.display import clear_output
# clear_output()

In [ ]:
# Checking to see if Cuda support is available 
# Output True = Cuda
# Output False = No Cuda (installing Cuda will be required to run the model on GPU)
import os 
import torch
print(torch.cuda.is_available())


Hozd létre a Huggingface hozzáférési tokenedet: [Create your Huggingface Access Token](https://huggingface.co/settings/tokens)

Hozz létre egy új tokent  
Adj meg egy új nevet  
Válaszd ki az írási jogosultságokat  
Másold ki a tokent, és mentsd el biztonságos helyre


A következő Python kód két fő feladatot lát el: az `os` modul importálása és egy környezeti változó beállítása.

1. Az `os` modul importálása:
   - Az `os` modul a Pythonban lehetőséget nyújt az operációs rendszerrel való interakcióra. Segítségével különféle operációs rendszerhez kapcsolódó feladatokat végezhetünk, például környezeti változók elérése, fájlokkal és könyvtárakkal való munka stb.
   - Ebben a kódban az `os` modult az `import` utasítással importáljuk. Ez az utasítás elérhetővé teszi az `os` modul funkcionalitását az aktuális Python szkriptben.

2. Környezeti változó beállítása:
   - A környezeti változó egy olyan érték, amelyet az operációs rendszeren futó programok elérhetnek. Ez egy módja annak, hogy konfigurációs beállításokat vagy más információkat tároljunk, amelyeket több program is használhat.
   - Ebben a kódban egy új környezeti változó kerül beállításra az `os.environ` szótár segítségével. A szótár kulcsa `'HF_TOKEN'`, az értéket pedig a `HUGGINGFACE_TOKEN` változóból kapja.
   - A `HUGGINGFACE_TOKEN` változó közvetlenül a kódrészlet felett van definiálva, és egy sztring értéket kap, `"hf_**************"` formában, a `#@param` szintaxis használatával. Ez a szintaxis gyakran Jupyter notebookokban használatos, hogy lehetővé tegye a felhasználói bemenetet és a paraméterek konfigurálását közvetlenül a notebook felületén.
   - A `'HF_TOKEN'` környezeti változó beállításával az érték elérhetővé válik a program más részei vagy az operációs rendszeren futó más programok számára.

Összességében ez a kód importálja az `os` modult, és beállít egy `'HF_TOKEN'` nevű környezeti változót a `HUGGINGFACE_TOKEN` változóban megadott értékkel.


In [ ]:
import os
# set the Hugging Face Token from 
# add the Hugging Face Token to the environment variables
HUGGINGFACE_TOKEN = "Enter Hugging Face Key" #@param {type:"string"}
os.environ['HF_TOKEN']HUGGINGFACE_TOKEN

Ez a kódrészlet egy clear_output nevű függvényt definiál, amelyet arra használnak, hogy törölje az aktuális cella kimenetét Jupyter Notebookban vagy IPythonban. Nézzük meg részletesebben a kódot és értsük meg a működését:

A clear_output függvény egy paramétert fogad, amelynek neve wait, és ez egy logikai érték. Alapértelmezés szerint a wait értéke False. Ez a paraméter határozza meg, hogy a függvény várjon-e addig, amíg új kimenet elérhető, mielőtt törölné a meglévő kimenetet.

Maga a függvény arra szolgál, hogy törölje az aktuális cella kimenetét. Jupyter Notebookban vagy IPythonban, amikor egy cella kimenetet generál, például nyomtatott szöveget vagy grafikus ábrákat, az a cella alatt jelenik meg. A clear_output függvény lehetővé teszi, hogy ezt a kimenetet töröljük.

A függvény implementációja nincs megadva a kódrészletben, amit az ellipszis (...) jelez. Az ellipszis egy helyőrző, amely az aktuális kódot jelöli, amely a kimenet törlését végzi. A függvény implementációja valószínűleg a Jupyter Notebook vagy IPython API-val való interakciót foglalja magában, hogy eltávolítsa az aktuális cella meglévő kimenetét.

Összességében ez a függvény egy kényelmes módot biztosít az aktuális cella kimenetének törlésére Jupyter Notebookban vagy IPythonban, megkönnyítve a megjelenített kimenet kezelését és frissítését interaktív kódolási munkamenetek során.


In [ ]:
# Download Phi-3-mini-4k-instruct model & Whisper Tiny
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

#whisper for speech to text()
import whisper
select_model ="tiny" # ['tiny', 'base']
whisper_model = whisper.load_model(select_model)

#from IPython.display import clear_output
#clear_output()

Hajtsuk végre szövegfelolvasást (TTS) az Edge TTS szolgáltatás segítségével. Nézzük meg a releváns függvények implementációit egyenként:

1. `calculate_rate_string(input_value)`: Ez a függvény egy bemeneti értéket kap, és kiszámítja a TTS hang sebességét reprezentáló karakterláncot. A bemeneti érték a beszéd kívánt sebességét jelöli, ahol az 1 normál sebességet jelent. A függvény úgy számítja ki a sebesség karakterláncot, hogy kivon 1-et a bemeneti értékből, megszorozza 100-zal, majd meghatározza az előjelet annak alapján, hogy a bemeneti érték nagyobb vagy egyenlő-e 1-nél. A függvény a sebesség karakterláncot a következő formátumban adja vissza: "{sign}{rate}".

2. `make_chunks(input_text, language)`: Ez a függvény egy bemeneti szöveget és egy nyelvet kap paraméterként. A szöveget nyelvspecifikus szabályok alapján darabolja fel. Ebben az implementációban, ha a nyelv "English", a függvény minden pontnál (".") felosztja a szöveget, eltávolítva az elején vagy végén lévő szóközöket. Ezután minden darabhoz hozzáad egy pontot, és visszaadja a szűrt darabok listáját.

3. `tts_file_name(text)`: Ez a függvény fájlnevet generál a TTS hangfájlhoz a bemeneti szöveg alapján. Több átalakítást végez a szövegen: eltávolítja a végén lévő pontot (ha van), kisbetűssé alakítja, eltávolítja az elején és végén lévő szóközöket, és helyettesíti a szóközöket aláhúzásokkal. Ezután a szöveget legfeljebb 25 karakterre rövidíti (ha hosszabb), vagy az egész szöveget használja, ha üres. Végül a [`uuid`] modul segítségével véletlenszerű karakterláncot generál, és kombinálja a rövidített szöveggel, hogy létrehozza a fájlnevet a következő formátumban: "/content/edge_tts_voice/{truncated_text}_{random_string}.mp3".

4. `merge_audio_files(audio_paths, output_path)`: Ez a függvény több hangfájlt egyetlen hangfájlba egyesít. Egy hangfájlok útvonalait tartalmazó listát és egy kimeneti útvonalat kap paraméterként. A függvény inicializál egy üres [`AudioSegment`] objektumot, amelyet [`merged_audio`]-nak nevez. Ezután végigmegy minden hangfájl útvonalán, betölti a hangfájlt a `AudioSegment.from_file()` metódus segítségével a `pydub` könyvtárból, és hozzáadja az aktuális hangfájlt a [`merged_audio`] objektumhoz. Végül exportálja az egyesített hangot a megadott kimeneti útvonalra MP3 formátumban.

5. `edge_free_tts(chunks_list, speed, voice_name, save_path)`: Ez a függvény végrehajtja a TTS műveletet az Edge TTS szolgáltatás segítségével. Egy szövegdarabok listáját, a beszéd sebességét, a hang nevét és a mentési útvonalat kapja paraméterként. Ha a darabok száma nagyobb, mint 1, a függvény létrehoz egy könyvtárat az egyes darabok hangfájljainak tárolására. Ezután végigmegy minden darabon, összeállít egy Edge TTS parancsot a `calculate_rate_string()` függvény, a hang neve és a darab szövege segítségével, majd végrehajtja a parancsot az `os.system()` függvény segítségével. Ha a parancs végrehajtása sikeres, hozzáadja a generált hangfájl útvonalát egy listához. Miután feldolgozta az összes darabot, egyesíti az egyes hangfájlokat a `merge_audio_files()` függvény segítségével, és elmenti az egyesített hangot a megadott mentési útvonalra. Ha csak egy darab van, közvetlenül generálja az Edge TTS parancsot, és elmenti a hangot a mentési útvonalra. Végül visszaadja a generált hangfájl mentési útvonalát.

6. `random_audio_name_generate()`: Ez a függvény véletlenszerű hangfájlnév generálására szolgál a [`uuid`] modul segítségével. Véletlenszerű UUID-t generál, karakterlánccá alakítja, az első 8 karaktert veszi, hozzáadja a ".mp3" kiterjesztést, és visszaadja a véletlenszerű hangfájlnév.

7. `talk(input_text)`: Ez a függvény a fő belépési pont a TTS művelet végrehajtásához. Egy bemeneti szöveget kap paraméterként. Először ellenőrzi a bemeneti szöveg hosszát, hogy meghatározza, hosszú mondatról van-e szó (600 karakternél hosszabb vagy egyenlő). A hossz és a `translate_text_flag` változó értéke alapján meghatározza a nyelvet, és létrehozza a szövegdarabok listáját a `make_chunks()` függvény segítségével. Ezután generál egy mentési útvonalat a hangfájlhoz a `random_audio_name_generate()` függvény segítségével. Végül meghívja az `edge_free_tts()` függvényt a TTS művelet végrehajtásához, és visszaadja a generált hangfájl mentési útvonalát.

Összességében ezek a függvények együttműködnek, hogy feldarabolják a bemeneti szöveget, fájlnevet generáljanak a hangfájlhoz, végrehajtsák a TTS műveletet az Edge TTS szolgáltatás segítségével, és egyesítsék az egyes hangfájlokat egyetlen hangfájlba.


In [ ]:
#@title Edge TTS
def calculate_rate_string(input_value):
    rate = (input_value - 1) * 100
    sign = '+' if input_value >= 1 else '-'
    return f"{sign}{abs(int(rate))}"


def make_chunks(input_text, language):
    language="English"
    if language == "English":
      temp_list = input_text.strip().split(".")
      filtered_list = [element.strip() + '.' for element in temp_list[:-1] if element.strip() and element.strip() != "'" and element.strip() != '"']
      if temp_list[-1].strip():
          filtered_list.append(temp_list[-1].strip())
      return filtered_list


import re
import uuid
def tts_file_name(text):
    if text.endswith("."):
        text = text[:-1]
    text = text.lower()
    text = text.strip()
    text = text.replace(" ","_")
    truncated_text = text[:25] if len(text) > 25 else text if len(text) > 0 else "empty"
    random_string = uuid.uuid4().hex[:8].upper()
    file_name = f"/content/edge_tts_voice/{truncated_text}_{random_string}.mp3"
    return file_name


from pydub import AudioSegment
import shutil
import os
def merge_audio_files(audio_paths, output_path):
    # Initialize an empty AudioSegment
    merged_audio = AudioSegment.silent(duration=0)

    # Iterate through each audio file path
    for audio_path in audio_paths:
        # Load the audio file using Pydub
        audio = AudioSegment.from_file(audio_path)

        # Append the current audio file to the merged_audio
        merged_audio += audio

    # Export the merged audio to the specified output path
    merged_audio.export(output_path, format="mp3")

def edge_free_tts(chunks_list,speed,voice_name,save_path):
  # print(chunks_list)
  if len(chunks_list)>1:
    chunk_audio_list=[]
    if os.path.exists("/content/edge_tts_voice"):
      shutil.rmtree("/content/edge_tts_voice")
    os.mkdir("/content/edge_tts_voice")
    k=1
    for i in chunks_list:
      print(i)
      edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{i}" --write-media /content/edge_tts_voice/{k}.mp3'
      print(edge_command)
      var1=os.system(edge_command)
      if var1==0:
        pass
      else:
        print(f"Failed: {i}")
      chunk_audio_list.append(f"/content/edge_tts_voice/{k}.mp3")
      k+=1
    # print(chunk_audio_list)
    merge_audio_files(chunk_audio_list, save_path)
  else:
    edge_command=f'edge-tts  --rate={calculate_rate_string(speed)}% --voice {voice_name} --text "{chunks_list[0]}" --write-media {save_path}'
    print(edge_command)
    var2=os.system(edge_command)
    if var2==0:
      pass
    else:
      print(f"Failed: {chunks_list[0]}")
  return save_path

# text = "This is Microsoft Phi 3 mini 4k instruct Demo" Simply update the text variable with the text you want to convert to speech
text = 'This is Microsoft Phi 3 mini 4k instruct Demo'  # @param {type: "string"}
Language = "English" # @param ['English']
# Gender of voice simply change from male to female and choose the voice you want to use
Gender = "Female"# @param ['Male', 'Female']
female_voice="en-US-AriaNeural"# @param["en-US-AriaNeural",'zh-CN-XiaoxiaoNeural','zh-CN-XiaoyiNeural']
speed = 1  # @param {type: "number"}
translate_text_flag  = False
if len(text)>=600:
  long_sentence = True
else:
  long_sentence = False

# long_sentence = False # @param {type:"boolean"}
save_path = ''  # @param {type: "string"}
if len(save_path)==0:
  save_path=tts_file_name(text)
if Language == "English" :
  if Gender=="Male":
    voice_name="en-US-ChristopherNeural"
  if Gender=="Female":
    voice_name=female_voice
    # voice_name="en-US-AriaNeural"


if translate_text_flag:
  input_text=text
  # input_text=translate_text(text, Language)
  # print("Translateting")
else:
  input_text=text
if long_sentence==True and translate_text_flag==True:
  chunks_list=make_chunks(input_text,Language)
elif long_sentence==True and translate_text_flag==False:
  chunks_list=make_chunks(input_text,"English")
else:
  chunks_list=[input_text]
# print(chunks_list)
# edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
# from IPython.display import clear_output
# clear_output()
# from IPython.display import Audio
# Audio(edge_save_path, autoplay=True)

from IPython.display import clear_output
from IPython.display import Audio
if not os.path.exists("/content/audio"):
    os.mkdir("/content/audio")
import uuid
def random_audio_name_generate():
  random_uuid = uuid.uuid4()
  audio_extension = ".mp3"
  random_audio_name = str(random_uuid)[:8] + audio_extension
  return random_audio_name
def talk(input_text):
  global translate_text_flag,Language,speed,voice_name
  if len(input_text)>=600:
    long_sentence = True
  else:
    long_sentence = False

  if long_sentence==True and translate_text_flag==True:
    chunks_list=make_chunks(input_text,Language)
  elif long_sentence==True and translate_text_flag==False:
    chunks_list=make_chunks(input_text,"English")
  else:
    chunks_list=[input_text]
  save_path="/content/audio/"+random_audio_name_generate()
  edge_save_path=edge_free_tts(chunks_list,speed,voice_name,save_path)
  return edge_save_path


edge_save_path=talk(text)
Audio(edge_save_path, autoplay=True)

A két függvény, a convert_to_text és a run_text_prompt megvalósítása, valamint két osztály, a str és az Audio deklarációja.

A convert_to_text függvény egy audio_path bemenetet kap, és egy whisper_model nevű modell segítségével átalakítja az audiót szöveggé. A függvény először ellenőrzi, hogy a gpu jelző True értékre van-e állítva. Ha igen, akkor a whisper_model bizonyos paraméterekkel kerül használatra, mint például word_timestamps=True, fp16=True, language='English', és task='translate'. Ha a gpu jelző False, akkor a whisper_model fp16=False paraméterrel kerül használatra. Az eredményül kapott átiratot egy 'scan.txt' nevű fájlba menti, és szövegként visszaadja.

A run_text_prompt függvény egy message és egy chat_history bemenetet kap. A phi_demo függvényt használja arra, hogy egy chatbot válaszát generálja az adott üzenet alapján. A generált választ továbbadja a talk függvénynek, amely az üzenetet audiófájllá alakítja, és visszaadja az audiófájl elérési útját. Az Audio osztályt használják az audiófájl megjelenítésére és lejátszására. Az audiót az IPython.display modul display függvényével jelenítik meg, és az Audio objektum autoplay=True paraméterrel kerül létrehozásra, így az audió automatikusan elindul. A chat_history frissül az üzenettel és a generált válasszal, majd egy üres string és a frissített chat_history kerül visszaadásra.

A str osztály egy beépített Python osztály, amely karakterek sorozatát reprezentálja. Számos metódust biztosít a szövegek manipulálására és kezelésére, mint például capitalize, casefold, center, count, encode, endswith, expandtabs, find, format, index, isalnum, isalpha, isascii, isdecimal, isdigit, isidentifier, islower, isnumeric, isprintable, isspace, istitle, isupper, join, ljust, lower, lstrip, partition, replace, removeprefix, removesuffix, rfind, rindex, rjust, rpartition, rsplit, rstrip, split, splitlines, startswith, strip, swapcase, title, translate, upper, zfill, és még sok más. Ezek a metódusok lehetővé teszik olyan műveletek végrehajtását, mint keresés, csere, formázás és szövegek manipulálása.

Az Audio osztály egy egyedi osztály, amely egy audió objektumot reprezentál. A Jupyter Notebook környezetben audiólejátszó létrehozására használják. Az osztály különböző paramétereket fogad el, mint például data, filename, url, embed, rate, autoplay, és normalize. A data paraméter lehet numpy tömb, minták listája, fájlnév vagy URL-t reprezentáló string, vagy nyers PCM adat. A filename paraméter egy helyi fájlt ad meg, amelyből az audióadatokat be kell tölteni, míg az url paraméter egy URL-t ad meg, ahonnan az audióadatokat le kell tölteni. Az embed paraméter meghatározza, hogy az audióadatokat beágyazott adat URI-ként vagy az eredeti forrásból kell-e hivatkozni. A rate paraméter az audióadatok mintavételi sebességét adja meg. Az autoplay paraméter meghatározza, hogy az audió automatikusan elinduljon-e. A normalize paraméter azt határozza meg, hogy az audióadatokat normalizálni (átméretezni) kell-e a maximális lehetséges tartományra. Az Audio osztály olyan metódusokat is biztosít, mint a reload, amely újratölti az audióadatokat fájlból vagy URL-ből, valamint olyan attribútumokat, mint a src_attr, autoplay_attr, és element_id_attr, amelyek az audió HTML elem megfelelő attribútumait adják vissza.

Összességében ezek a függvények és osztályok arra szolgálnak, hogy audiót szöveggé alakítsanak, chatbot válaszokat generáljanak audió formájában, és audiót jelenítsenek meg és játsszanak le a Jupyter Notebook környezetben.


In [ ]:
#@title Run gradio app
def convert_to_text(audio_path):
  gpu=True
  if gpu:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=True,language='English',task='translate')
  else:
    result = whisper_model.transcribe(audio_path,word_timestamps=True,fp16=False,language='English',task='translate')
  with open('scan.txt', 'w') as file:
    file.write(str(result))
  return result["text"]


import gradio as gr
from IPython.display import Audio, display
def run_text_prompt(message, chat_history):
    bot_message = phi_demo(message)
    edge_save_path=talk(bot_message)
    # print(edge_save_path)
    display(Audio(edge_save_path, autoplay=True))

    chat_history.append((message, bot_message))
    return "", chat_history


def run_audio_prompt(audio, chat_history):
    if audio is None:
        return None, chat_history
    print(audio)
    message_transcription = convert_to_text(audio)
    _, chat_history = run_text_prompt(message_transcription, chat_history)
    return None, chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chat with Phi 3 mini 4k instruct")

    msg = gr.Textbox(label="Ask anything")
    msg.submit(run_text_prompt, [msg, chatbot], [msg, chatbot])

    with gr.Row():
        audio = gr.Audio(sources="microphone", type="filepath")

        send_audio_button = gr.Button("Send Audio", interactive=True)
        send_audio_button.click(run_audio_prompt, [audio, chatbot], [audio, chatbot])

demo.launch(share=True,debug=True)


---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
